# Twitter Query Expansion
© Jason Pyanowski

In this demo file, the application of the project **Twitter Query Expansion** is explained. Starting with the initial data retrieval and download of the Word Embedding models. Then the pipeline is invoked and the results are inspected. 

## Data Preparation
 This allows to search Tweets quickly. In order to make use of this, the Tweets must be parsed from the PostgreSQL database into an Elastic Search Index. This task is handled by the script `/scripts/tweet_feeder.py` as stated below. 

In [1]:
from pipeline.text_processor import TextProcessor
a = TextProcessor()
a.nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   morphologizer     token.morph                      pos_acc            False      
                      token.pos                        morph_acc                     
                                                       morph_per_feat                
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_l

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'morphologizer': {'assigns': ['token.morph', 'token.pos'],
   'requires': [],
   'scores': ['pos_acc', 'morph_acc', 'morph_per_feat'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'hashtag_matcher': {'assigns': [],
   'requires': [],
   'scores': [],


In [ ]:
!python3 scripts/tweet_feeder.py -i tweet_index -t tweet_table -wc 30

## Download Models
This Pipeline allows tu use different word embedding models. The download link of the desired model can be used to load the model below. The model types of `fasttext` and `word2vec` are currently supported. To speed up the performance of the query expansion pipeline, the models are consequently compressed.

|Parameter|Possible Values|Datatype|
|---|---|---|
|type|`'fasttext'`, `'word2vec'`|`str`|
|url|`'url to model'`|`str`|

Download Word2Vec model

In [ ]:
from scripts.model_loader import load_model
load_model(type="word2vec", url="https://cloud.devmount.de/d2bc5672c523b086/german.model")

Download FastText model

In [ ]:
load_model(type="fasttext", url="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz")

## Define Queries
Specify queries on which to evaluate the pipeline. Queries may include Twitter-specific syntax like hashtags `#EU` or user mentions `@bundestag`.

In [1]:
QUERIES = [
    "große Koalition gescheitert unter Merkel? #Groko #SPD #CDU",
    "Lauterbach Deutschland Corona-Maßnahmen #Impfung",
    "@bundestag Bundestagswahl 2021 Ergebnisse",
    "EU Brexit Boris Johnson",
    "Gesetzliche Rentenversicherung Rente Mit 67",
    "Klimapolitik Deutschland #Grüne",
    "Asylpolitik Merkel",
    "Soli abschaffen Westen",
    "Bundeswehr Afghanistan Krieg stoppen",
    "Deutschland Energiewende mit SPD CDU"
]

## Set Word Embedding Parameters
In order to obtain the desired results, modify the parameters for Word Embeddings. These configurations determine which of the initial query terms are actually used to find related terms.

| Parameter | Possible Values | Datatype |
|---|---|---|
|type|`'word2vec', 'fasttext'`|`str`|
|model| `'path to model'`|`str`|
|pos_list|`['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'CONJ', 'DET', 'EOL', 'IDS', 'INTJ', 'NAMES', 'NOUN', 'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SPACE', 'SYM', 'VERB', 'X']`| `list[str]`|
|entity_list|`['LOC', 'MISC', 'ORG', 'PER']`|`list[str]`|
|hashtag|`True, False`|`bool`|
|user|`True, False`|`bool`|
|num_nearest_terms|`1...N`|`int`|


In [2]:
EMBEDDING_PARAMS = {
    "type": "fasttext",
    "model": "models/fasttext/cc.de.300.model",
    "pos_list": ["NOUN","ADJ","VERB","PROPN"],
    "entity_list": ['LOC', 'ORG'],
    "hashtag": True,
    "user": False,
    "num_nearest_terms": 5
}

## Set Elastic Search Parameters
After Query Expansion, the Tweets are retrieved from an Elastic Search Index. Specify the parameters below and make sure that an Index is running on your machine. 

| Parameter | Possible Values | Datatype |
|---|---|---|
|index|`'tweets'`|`str`|
|num_of_tweets|`1...N`| `int`|
|retweet|`True, False`|`bool`|
|hashtag_boost|`0...N`|`float`|
|tweet_range|`(date, date)`|`tuple`|

In [3]:
ELASTIC_PARAMS = {
    "index": "tweets_all",
    "num_of_tweets": 10,
    "retweet": False,
    "hashtag_boost": 1.0,
    "tweet_range": ("2020-01-01", "2023-01-01")
}

## Execute Pipeline
Run the Pipeline - the results are stored in the `/out` directory.

In [4]:
# run pipeline
from scripts import pipeline

res = pipeline.run(
    queries=QUERIES, 
    embedding_params=EMBEDDING_PARAMS,
    elastic_params=ELASTIC_PARAMS)

Processing text using SpaCy...
Evaluating fasttext model...


AttributeError: 'str' object has no attribute 'text'

## Inspect Results
Load results and have a look through the retrieved Tweets. 

In [ ]:
for tweets, i in zip(res,range(len(QUERIES))):
    print("Query:", QUERIES[i], "\n")
    for tweet in tweets["tweets"]:
        print("->", tweet["_source"]["txt"])